# Complex Solution SymPy Code Generation
## Solutions with Cation Ordering
Generation of configurational entropy (${S^{config}}$) using combinatorics coupled with $n^{th}$-order Taylor expansion of the non-configurational Gibbs free energy (${G^*}$).  The final expression for the Gibbs free energy of solution is given by $G = -{T}{S^{config}} + {G^*}$.  

This notebook illustrates construction of this problem using the coder module of the thermoengine package.  

Generally, the Taylor expansion of ${G^*}$ is taken to order two (equivalent to regular solution theory) and cation-ordering between symmetrically non-equivalent crystallographic sites is assumed to be non-convergent, i.e. the random ordering state is not acheived at finite temperature.  Alternately, cation-ordering may be modeled as convergent, inducing a symmetry breaking phase transition at finite temperature, which necessitates Taylor expansion of ${G^*}$ to at least $4^{th}$ order (in ordering parameter) with retention of only even powers of the ordering variable(s) in the expansion.  

This notebook illustrates non-convergent ordering in a reciprocal solution model for orthpyroxene in the compositional space of the pyroxene quadrilateral: Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>-Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>-CaMgSi<sub>2</sub>O<sub>6</sub>-CaFeSi<sub>2</sub>O<sub>6</sub>

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
sym.init_printing()
from thermoengine import coder
from thermoengine import core

## Complex Solution Properties - General structure of the model
There are three terms:
- Terms describing standard state contributions
- Terms describing the configurational entropy of solution
- Terms describing the excess enthalpy of solution  

Assumptions:
- There are $c$ components in the system
- There may be more endmember species, $w$, than there are components, thereby allowing for reciprocal solutions
- Cation ordering is permitted, which may be either convergent or non-convergent.  There may be zero or more cation ordering variables, $s$.
- The configurational entropy formulation assumes random mixing on symmetrically distinct crystallographic sites
- The excess enthalpy is described using a Taylor series expansion in compositional and ordering variables.  The order of the expansion is $\nu$. 

## Number of solution components and number of solution species
Note, that the example illustrated in this notebook - orthopyroxene in the system Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>-Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>-CaMgSi<sub>2</sub>O<sub>6</sub>-CaFeSi<sub>2</sub>O<sub>6</sub>, requires three endmember thermodynamic components but clearly has four endmember species.  This is an example of a recipocal solution. One of the species endmembers is compositionally redundent, but *not* energetically redundant.  Hence the Gibbs free energy change of the reaction:  
Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> + 2 CaFeSi<sub>2</sub>O<sub>6</sub> = Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> + 2 CaMgSi<sub>2</sub>O<sub>6</sub>  
is not zero, even though the concentration of the species Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> may be expressed as:  
2 CaFeSi<sub>2</sub>O<sub>6</sub> - 2 CaMgSi<sub>2</sub>O<sub>6</sub> + Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>  
Additionally, there is one variable that denotes the degree of cation ordering of Fe<sup>++</sup> and Mg over the M1 and M2 crystallographic sites in the pyroxene structure.

In [ ]:
nc = 3
nw = 4
ns = 1

## Create a complex solution model
A *complex* solution is one that includes ordering parameters as well as endmember thermodynamic components.  
Instantiate the class with the specified number of endmember thermodynamic components and species

In [ ]:
model = coder.ComplexSolnModel(nc=nc, ns=ns, nw=nw)

## Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution
- $s$ is a vector of ordering parameters

### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of thermodynamic components in the system

### and a reduced set of independent composition variables
- $r$ is a vector of independent mole fractions in the system. By convention, $r_{i-1}=X_i$, where $i$ ranges from the second index of $X$ up to $c$.  Hence the length of the vector $r$ is $c-1$

In [ ]:
n = model.n
nT = model.nT
s = model.s
X = n/nT
r = X[1:]
n, nT, X, r, s

## Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu^o$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
T,P,mu

## Define the standard state contribution to solution properties

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

## Define configurational entropy and configurational Gibbs free energy
Configurational enropy is calculated by counting site configurations, that is the number of ways of mixing Fe<sup>++</sup>, Mg and Ca on the M2 site, $\Omega^{M2}$,  times the number of ways of mixing Fe<sup>++</sup> and Mg on the M1 site, $\Omega^{M1}$; configurations ($\Omega$) equal  $\Omega^{M1}\Omega^{M2}$.  The assumption is made that the mixing on each site is random, i.e.  
If there are two cations on site M1, and their mole fractions on that site are denoted $X$ and $Y$, and if there is one such sites in the formula unit, then the number of configurations, $\Omega$, associted with ***random*** mixing of cations on that site is:    
$\Omega  = \left[ {\frac{{\left( {X + Y} \right)!}}{{X!Y!}}} \right]$  
and the molar configurational entropy conribution associated with these configurations is given by Boltzmann's law: ${{\hat S}^{conf}} =  R\log \Omega$:  
${{\hat S}^{conf}} =  cR\log \left[ {\frac{{\left( {X + Y} \right)!}}{{X!Y!}}} \right]$  
Using Stirlings approximation for large factorials, $\log X! = X\log X - X$, the configurational entropy can be written:  
${{\hat S}^{conf}} =  cR\left[ - {X\log X - Y\log Y + \left( {X + Y} \right)\log \left( {X + Y} \right)} \right]$  

Consequently, to utilize this appropach we must define site mole fractions in terms of our chosen set of independent compositional variables and ordering parameters.
#### There are 5 site mole fractions:  
$X_{Ca}^{M2}$, $X_{Mg}^{M2}$, $X_{{Fe}^{2+}}^{M2}$, $X_{Mg}^{M1}$, $X_{{Fe}^{2+}}^{M1}$  
#### The requirement of filled sites requires:
1. $X_{Ca}^{M2}$ + $X_{Mg}^{M2}$ + $X_{{Fe}^{2+}}^{M2}$ = 1
2. $X_{Mg}^{M1}$ + $X_{{Fe}^{2+}}^{M1}$ =1

#### Asuuming the endmembers are ordered as:
- $n_1$, $X_1$, CaMgSi<sub>2</sub>O<sub>6</sub>
- $n_2$, $X_2$, CaFeSi<sub>2</sub>O<sub>6</sub>
- $n_3$, $X_3$, Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>

#### There are two independent compositional variables:
- CaMgSi<sub>2</sub>O<sub>6</sub> = $1-r_1-r_2$
- CaFeSi<sub>2</sub>O<sub>6</sub> = $r_1$
- Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> = $r_2$  

#### The requirement of mass balance requires:
3. $r_1$ = $X_{{Fe}^{2+}}^{M2}$ + $X_{{Fe}^{2+}}^{M1}$
4. $r_2$ = 1 - $X_{Ca}^{M2}$

#### There is one ordering parameter:
5. $s_1$ = $X_{{Fe}^{2+}}^{M2}$ - $X_{Mg}^{M2}$

#### Relations 1-5 may be solved simultaneously to give the following site mole fraction definitions:
- $X_{Ca}^{M2}$ = $1-r_2$
- $X_{Mg}^{M2}$ = $\frac{r_2-s_1}{2}$
- $X_{{Fe}^{2+}}^{M2}$ = $\frac{r_2+s_1}{2}$
- $X_{Mg}^{M1}$ = $1-r_1+\frac{r_2+s_1}{2}$
- $X_{{Fe}^{2+}}^{M1}$ = $r_1-\frac{r_2+s_1}{2}$  

While this system is fairly easy to solve by inspection, for more complex situations, assemble the relations in a list of equations that evaluate to zero, and automatiocally solve that system of equations using the sympy routine linsolve, i.e.  
```
system = [xCaM2 + xMgM2 + xFeM2 - 1, xMgM1 + xFeM1 - 1, xFeM2 + xFeM1 - r[0], 1 - xCaM2 - r[1],
          xFeM2 - xMgM2 - s[0]]
ans = sym.linsolve(system, xCaM2, xMgM2, xFeM2, xMgM1, xFeM1)
```

In [ ]:
xCaM2 = 1 - r[1]
xMgM2 = (r[1]-s[0])/2
xFeM2 = (r[1]+s[0])/2
xMgM1 = 1 - r[0] + (r[1]+s[0])/2
xFeM1 = r[0] - (r[1]+s[0])/2

### The following functions implement random mixing configurational entropy on the M1 and M2 sites: 

In [ ]:
def Sconf_M1_random(X, Y):
    A = X*sym.log(X) - X
    B = Y*sym.log(Y) - Y
    ApB = (X+Y)*sym.log(X+Y) - (X+Y)
    return ApB - A - B
def Sconf_M2_random(X, Y, Z):
    A = X*sym.log(X) - X
    B = Y*sym.log(Y) - Y
    C = Z*sym.log(Z) - Z
    ApBpC = (X+Y+Z)*sym.log(X+Y+Z) - (X+Y+Z)
    return ApBpC - A - B - C

## Configurational entropy
$R$ is the gas constant

In [ ]:
R = sym.symbols('R')
S_config = Sconf_M1_random(xMgM1, xFeM1) + Sconf_M2_random(xCaM2, xMgM2, xFeM2)
S_config *= R*nT
S_config

## Configurational Gibbs free energy
Note that this quantity is extensive, with units of J, *not J/mole*

In [ ]:
G_config = -T*S_config
G_config

## $\hat G^*$ - Non-configurational molar Gibbs free energy
$\hat G^*$ includes all standard state and excess Gibbs free energy contributions.  It is generally modeled as a Taylor expansion in composition ($r$) and ordering ($s$) variables of order 2, 3 or 4.  Here, we choose a model of order two.
#### Taylor expansion of $\hat G^*$
For a second order expansion, the number of Taylor expansion coefficients is:
- 1 for $G_{0}$
- nc-1 for $G_{r_i}$, $i=1...nc-1$
- ns for $G_{s_i}$, $i=1...ns$
- (nc-1)(nc-2)/2 for $G_{{r_i},{r_{i+1}}}$, $i= 1...nc-2$
- ns(ns-1)/2 for $G_{{s_i},{s_{i+1}}}$, $i= 1...ns-1$
- ns(nc-1) for $G_{{r_i},{s_j}}$, $i= 1...nc-1$, $j=1...ns$
- nc-1 for $G_{{r_i},{r_i}}$, $i= 1...nc-1$
- ns for $G_{{s_i},{s_i}}$, $i= 1...ns$

In [ ]:
(count, taylor, taylor_coeff, taylor_terms) = model.taylor_expansion()
print ('Number of Taylor expansion terms = {0:.0f}'.format(count))
taylor

### Identify Taylor terms of $\hat{G}^*$ corresponding to component endmembers:
1. diopside, CaMgSi<sub>2</sub>O<sub>6</sub>
2. hedenbergite, CaFeSi<sub>2</sub>O<sub>6</sub>
3. enstatite, Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>

In [ ]:
eqn1 = model.eval_endmember([1,0,0],[0],taylor) - mu[0]
eqn2 = model.eval_endmember([0,1,0],[0],taylor) - mu[1]
eqn3 = model.eval_endmember([0,0,1],[-1],taylor) - mu[2]
params = []
units = []
symparams = []
eqn1, eqn2, eqn3

### Identify Taylor terms of $\hat{G}^*$ corresponding to dependent species endmembers:
ferrosilite, Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>

In [ ]:
gFs = model.eval_endmember([-2,2,1],[1],taylor)
gFs

### Identify the free energy of the reciprocal reaction between endmember species:
Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> + 2 CaFeSi<sub>2</sub>O<sub>6</sub> = Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> + 2 CaMgSi<sub>2</sub>O<sub>6</sub>  
is defined as the "reciprocal energy," $F$, denoting the non-co-planarity of the non-configurational Gibbs free energy of the endmember species. In general, all reciprocal solutions have non-zero $F$. In the paper on pyroxene thermodynamics by Sack and Ghiorso (Contributions to Mineralogy and Petrology 116: 277-286, 1994) $F$ is notated as $\Delta \bar G_{27}^o$  

$F$ is conveniently defined in terms of expressions 1-4:

In [ ]:
Fh,Fs,Fv = sym.symbols('Fh Fs Fv')
params.append('Fh')
units.append('J/mol')
symparams.append(Fh)
params.append('Fs')
units.append('J/K-mol')
symparams.append(Fs)
params.append('Fv')
units.append('J/bar-mol')
symparams.append(Fv)
F = Fh - T*Fs + P*Fv
eqn4  =   model.eval_endmember([-2,2,1],[ 1],taylor)
eqn4 += 2*model.eval_endmember([ 1,0,0],[ 0],taylor)
eqn4 -=   model.eval_endmember([ 0,0,1],[-1],taylor)
eqn4 -= 2*model.eval_endmember([ 0,1,0],[ 0],taylor)
eqn4 -= F
eqn4

### Identify the free energy of the ordering reaction:
MgFeSi<sub>2</sub>O<sub>6</sub> = FeMgSi<sub>2</sub>O<sub>6</sub>  
which will be notated as $Gex$, (in Sack and Ghiorso, 1994, $\Delta \bar G_{EX}^o$)  
Note that both compositions, MgFeSi<sub>2</sub>O<sub>6</sub> and FeMgSi<sub>2</sub>O<sub>6</sub>, are equivalent and defined by CaFeSi<sub>2</sub>O<sub>6</sub> - CaMgSi<sub>2</sub>O<sub>6</sub> + Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>. They differ only by the sign of the ordering parameter.

In [ ]:
Hex,Vex = sym.symbols('Hex Vex')
params.append('Hex')
units.append('J/mol')
symparams.append(Hex)
params.append('Vex')
units.append('J/bar-mol')
symparams.append(Vex)
Gex = Hex + P*Vex
eqn5  = model.eval_endmember([-1,1,1],[ 1],taylor)
eqn5 -= model.eval_endmember([-1,1,1],[-1],taylor)
eqn5 -= Gex
eqn5

### Identify the free energy of the reciprocal ordering reaction:
Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> + Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> = MgFeSi<sub>2</sub>O<sub>6</sub> + FeMgSi<sub>2</sub>O<sub>6</sub>  
which will be notated as $Gx$, (in Sack and Ghiorso, 1994, $\Delta \bar G_{X}^o$) 

In [ ]:
Hx,Vx = sym.symbols('Hx Vx')
params.append('Hx')
units.append('J/mol')
symparams.append(Hx)
params.append('Vx')
units.append('J/bar-mol')
symparams.append(Vx)
Gx = Hx + P*Vx
eqn6  = model.eval_endmember([-1,1,1],[ 1],taylor)
eqn6 += model.eval_endmember([-1,1,1],[-1],taylor)
eqn6 -= model.eval_endmember([ 0,0,1],[-1],taylor)
eqn6 -= model.eval_endmember([-2,2,1],[ 1],taylor)
eqn6 -= Gx
eqn6

### Identify regular solution interaction parameters:
- Ca-Mg interaction on the M2 site, the join Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - CaMgSi<sub>2</sub>O<sub>6</sub>, denoted WM2CaMg
- Ca-Fe interaction on the M2 site, the join Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - CaFeSi<sub>2</sub>O<sub>6</sub>, denoted WM2CaFe
- Fe-Mg interaction on the M1 site, the joins Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - MgFeSi<sub>2</sub>O<sub>6</sub> or Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - FeMgSi<sub>2</sub>O<sub>6</sub> or CaMgSi<sub>2</sub>O<sub>6</sub> - CaFeSi<sub>2</sub>O<sub>6</sub>, which are assumed to be energetically equivalent, denoted WM1FeMg (in Sack and Ghiorso, 1994, $W_{12}$
- Fe-Mg interaction on the M2 site, the joins FeMgSi<sub>2</sub>O<sub>6</sub> - Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> or Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - MgFeSi<sub>2</sub>O<sub>6</sub>, which are assumed to be energetically equivalent, denoted WM2FeMg  

Along the A-B join, described using a regular solution parameter, $W$, $\hat G^*$ is given by  
${\hat G}^*(X_A,X_B)={X_A}{\hat G}^*(A)+{X_B}{\hat G}^*(B)+W{X_A}{X_B}$, so   
$W = \frac{{\hat G}^*(X_A,X_B) - {X_A}{\hat G}^*(A) - {X_B}{\hat G}^*(B)}{{X_A}{X_B}}$  
Taking the midpoint of the join provides a way to define the parameter:  
$W = \frac{{\hat G}^*(\frac{1}{2},\frac{1}{2}) - {\frac{1}{2}}{\hat G}^*(A) - {\frac{1}{2}}{\hat G}^*(B)}{{\frac{1}{2}}{\frac{1}{2}}} = 4{\hat G}^*(\frac{1}{2},\frac{1}{2}) - 2{\hat G}^*(A) - 2{\hat G}^*(B)$

- Ca-Mg interaction on the M2 site, the join Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - CaMgSi<sub>2</sub>O<sub>6</sub>, denoted WM2CaMg

In [ ]:
WhM2CaMg,WvM2CaMg = sym.symbols('WhM2CaMg WvM2CaMg')
params.append('WhM2CaMg')
units.append('J/mol')
symparams.append(WhM2CaMg)
params.append('WvM2CaMg')
units.append('J/bar-mol')
symparams.append(WvM2CaMg)
WM2CaMg = WhM2CaMg + P*WvM2CaMg
eqn7 = model.eval_regular_param([1,0,0],[0],[0,0,1],[-1],taylor) - WM2CaMg
eqn7

- Ca-Fe interaction on the M2 site, the join Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - CaFeSi<sub>2</sub>O<sub>6</sub>, denoted WM2CaFe

In [ ]:
WhM2CaFe,WvM2CaFe = sym.symbols('WhM2CFe WvM2CaFe')
params.append('WhM2CaFe')
units.append('J/mol')
symparams.append(WhM2CaFe)
params.append('WvM2CaFe')
units.append('J/bar-mol')
symparams.append(WvM2CaFe)
WM2CaFe = WhM2CaFe + P*WvM2CaFe
eqn8 = model.eval_regular_param([0,1,0],[0],[-2,2,1],[1],taylor) - WM2CaFe
eqn8

- Fe-Mg interaction on the M1 site, the joins Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - MgFeSi<sub>2</sub>O<sub>6</sub> or Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - FeMgSi<sub>2</sub>O<sub>6</sub> or CaMgSi<sub>2</sub>O<sub>6</sub> - CaFeSi<sub>2</sub>O<sub>6</sub>, which are assumed to be energetically equivalent, denoted WM1FeMg (in Sack and Ghiorso, 1994, $W_{12}$

In [ ]:
WhM1FeMg,WvM1FeMg = sym.symbols('WhM1FeMg WvM1FeMg')
params.append('WhM1FeMg')
units.append('J/mol')
symparams.append(WhM1FeMg)
params.append('WvM1FeMg')
units.append('J/bar-mol')
symparams.append(WvM1FeMg)
WM1FeMg = WhM1FeMg + P*WvM1FeMg
eqn9 = model.eval_regular_param([1,0,0],[0],[0,1,0],[0],taylor) - WM1FeMg
eqn9

- Fe-Mg interaction on the M2 site, the joins FeMgSi<sub>2</sub>O<sub>6</sub> - Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> or Fe<sub>2</sub>Si<sub>2</sub>O<sub>6</sub> - MgFeSi<sub>2</sub>O<sub>6</sub>, which are assumed to be energetically equivalent, denoted WM2FeMg

In [ ]:
WhM2FeMg,WvM2FeMg = sym.symbols('WhM2FeMg WvM2FeMg')
params.append('WhM2FeMg')
units.append('J/mol')
symparams.append(WhM2FeMg)
params.append('WvM2FeMg')
units.append('J/bar-mol')
symparams.append(WvM2FeMg)
WM2FeMg = WhM2FeMg + P*WvM2FeMg
eqn10 = model.eval_regular_param([-1,1,1],[1],[0,0,1],[-1],taylor) - WM2FeMg
eqn10

### Solve for the Taylor coefficients in terms of the preferred parameters

In [ ]:
system = [eqn1, eqn2, eqn3, eqn4, eqn5, eqn6, eqn7, eqn8, eqn9, eqn10]
system

In [ ]:
taylor_soln = sym.linsolve(system, taylor_coeff).args[0]
taylor_soln

### Substitute terms into $\hat G^*$

In [ ]:
sub_list = []
for a,b in zip(taylor_coeff,taylor_soln):
    sub_list.append((a,b))
G_star_molar = taylor.subs(sub_list)

In [ ]:
print(params)
print(units)
print(symparams)

## Define the Gibbs free energy of solution

In [ ]:
G = G_config + nT*G_star_molar
G

## Find the condition of homogeneous equilibrium:
$\frac{{\partial \hat G*}}{{\partial {s_1}}} = 0$

In [ ]:
dgds = (nT*G_star_molar+G_config).diff(s[0]).simplify()
dgds

## Identify bounds on the ordering parameter 
The code generated to implement this model must compute numerical values of the ordering parameter as a functrion of compositrion, temperature and pressure.  This task requires an iterative procedure.  To construct this procedure the model must have information on the permissble domain of the ordering parameter.  

Values of the ordering parameter are bounded by the composition of the solution. We contrain all site mole fractions to have values in the range 0 to 1, and solve this system of inequality constraints to obtain a logical expression that embodies the feasible domain for the numerical procedure. 

In [ ]:
out = sym.reduce_inequalities(inequalities=[
    0 <= (r[1]-s[0])/2, (r[1]-s[0])/2 <= 1,
    0 <= (r[1]+s[0])/2, (r[1]+s[0])/2 <= 1, 
    0 <= 1-r[0]+(r[1]+s[0])/2,  1-r[0]+(r[1]+s[0])/2 <= 1, 
    0 <= r[0]-(r[1]+s[0])/2, r[0]-(r[1]+s[0])/2 <= 1], symbols=[s[0]])
out

## Add the Gibbs free energy of solution to the model

In [ ]:
model.add_expression_to_model(G, list(zip(params, units, symparams)), ordering_functions=([dgds],s,[0],out))

... give the model a unqiue name

In [ ]:
model.module = "Complex_Solution"

... assign a formula string for code generation  
... assign a conversion string to map element concentrations to moles of end members
... assign a test string to evaluate the feasibility of input compositions

In [ ]:
model.formula_string = 'Ca[Ca]Mg[Mg]Fe[Fe]Si[Si]O6'
model.conversion_string = ['[0]=[Ca]-[Fe]', '[1]=[Fe]', '[2]=-0.5*[Ca]+0.5*[Fe]+0.5*[Mg]']
model.test_string = ['[0]+[1] > 0.0', '[1] > 0.0', '[0]+2.0*[2] > 0.0']

# Define Parameters of an Orthopyroxene Solution
Components
1. diopside, CaMgSi<sub>2</sub>O<sub>6</sub>
2. hedenbergite, CaFeSi<sub>2</sub>O<sub>6</sub>
3. enstatite, Mg<sub>2</sub>Si<sub>2</sub>O<sub>6</sub>

Original calibration from Sack and Ghiorso (Contributions to Mineralogy and Petrology, 116:287-300, 1994):
```
F       = -13807 + 2.319*T - 0.05878*P;  /* joules     */
Gex     =  -7824           - 0.1213*P;   /* joules/K   */
Gx      =  -1883           + 0.02824;    /* joules/bar */
WM2CaMg =  31631           + 0.03347*P;  /* joules     */
WM2CaFe =  17238           + 0.04602*P;  /* joules/K   */
WM1FeMg =   8368           + 0.01412*P;  /* joules/bar */
WM2FeMg =   8368           + 0.01412*P;  /* joules     */
```
Asymmetry along the Ca-Mg and Ca-Fe joins (considered by Sack and Ghiorso, 1994) is not considered in this example in order to simplify the presentation.

In [ ]:
print (params)
paramValues = {'Fh':-13807.0, 'Fs':-2.319, 'Fv':-0.05878, \
               'Hex':-7824.0, 'Vex':-0.1213, \
               'Hx':-1883.0, 'Vx':0.02824, \
               'WhM2CaMg':31631.0, 'WvM2CaMg':0.03347, \
               'WhM2CaFe':17238.0, 'WvM2CaFe':0.04602, \
               'WhM1FeMg':8368.0, 'WvM1FeMg':0.01412, \
               'WhM2FeMg':8368.0, 'WvM2FeMg':0.01412, \
               'T_r':298.15, 'P_r':1.0}
print (paramValues)

Generate both fast computation and calibibration code for the feldspar solution

# Use code printers to construct "C" package code

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

## Choose model type and create model
model_type is "fast" or "calib"

In [ ]:
model_type = "calib"

In [ ]:
model.create_code_module(phase="Orthopyroxene", params=paramValues, 
                         endmembers=['Diopside_berman', 'Hedenbergite_berman', 'Enstatite_berman'], 
                         prefix="cy", module_type=model_type, silent=False, 
                         add_code_to_access_order_paramater=True)

## Load the module

In [ ]:
import Complex_Solution
%cd ..

## Test and time the generated functions for Orthopyroxene (T in K, P in bars)

In [ ]:
t = 2000.00
p = 1.0
n = np.array([1.1, 1.2, 1.3])

## Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_identifier())
    print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_name())
    print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_formula(t,p,n))
except AttributeError:
    pass
try:
    print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_identifier())
    print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_name())
    print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_formula(t,p,n))
except AttributeError:
    pass

Test intrinsic element conversion routine ...

In [ ]:
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,nc):
        end = Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_elm_to_moles(e)
    for i in range(0,nc):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Complex_Solution.cy_Orthopyroxene_Complex_Solution_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,nc):
        end = Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_elm_to_moles(e)
    for i in range(0,nc):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass

Test various conversion routines ...

In [ ]:
try:
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_moles_to_tot_moles(n))
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_moles_to_mole_frac(n))
    e = Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_moles_to_elm(n)
    print (e)
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_elm_to_moles(e))
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_elm_to_tot_moles(e))
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_conv_elm_to_tot_grams(e))
except AttributeError:
    pass
try:
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_moles_to_tot_moles(n))
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_moles_to_mole_frac(n))
    e = Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_moles_to_elm(n)
    print (e)
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_elm_to_moles(e))
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_elm_to_tot_moles(e))
    print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_conv_elm_to_tot_grams(e))
except AttributeError:
    pass

### Execute a method that retrieves the ordering parameter 
This method is normally hidden from the Phases module implementation.

In [ ]:
nn = np.array([-0.99, 1, 1])
print(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_formula(t,p,nn))
print("Kd = (XFeM1 XMgM2)/(XFeM2 XMgM1), s = XFeM2 - XMgM2")
for tc in [600.0, 900.0, 1000.0, 1200.0, 1400.0, 1600.0, 1800.0, 2000.0]:
    ss = Complex_Solution.cy_Orthopyroxene_Complex_Solution_order_params(tc+273.15,p,nn)
    vCaM2 = xCaM2.subs([(model.n[0],nn[0]),(model.n[1],nn[1]),(model.n[2],nn[2]),(s[0],ss[0])])
    vMgM2 = xMgM2.subs([(model.n[0],nn[0]),(model.n[1],nn[1]),(model.n[2],nn[2]),(s[0],ss[0])])
    vFeM2 = xFeM2.subs([(model.n[0],nn[0]),(model.n[1],nn[1]),(model.n[2],nn[2]),(s[0],ss[0])])
    vMgM1 = xMgM1.subs([(model.n[0],nn[0]),(model.n[1],nn[1]),(model.n[2],nn[2]),(s[0],ss[0])])
    vFeM1 = xFeM1.subs([(model.n[0],nn[0]),(model.n[1],nn[1]),(model.n[2],nn[2]),(s[0],ss[0])])
    K = vFeM1*vMgM2/(vMgM1*vFeM2)
    print ("T {0:8.2f}  s {1:8.4f}  RT ln(Kd) {2:10.3f} kJ".format(tc, ss[0], 8.3143*(tc+273.15)*(np.log(float(K))/1000.0))) 

### Execute the standard thermodynamic property retrieval functions:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
try:
    print(fmt.format('G', Complex_Solution.cy_Orthopyroxene_Complex_Solution_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Complex_Solution.cy_Orthopyroxene_Complex_Solution_s(t,p,n), 'J/K'))
    print(fmt.format('V', Complex_Solution.cy_Orthopyroxene_Complex_Solution_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Complex_Solution.cy_Orthopyroxene_Complex_Solution_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Complex_Solution.cy_Orthopyroxene_Complex_Solution_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Complex_Solution.cy_Orthopyroxene_Complex_Solution_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Complex_Solution.cy_Orthopyroxene_Complex_Solution_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Complex_Solution.cy_Orthopyroxene_Complex_Solution_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Complex_Solution.cy_Orthopyroxene_Complex_Solution_Kp(t,p,n), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_s(t,p,n), 'J/K'))
    print(fmt.format('V', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_Kp(t,p,n), ''))
except AttributeError:
    pass

### Execute functions that access endmember properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of components", Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_formula(index)))
        print ("mw: {0:10.2f}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_mw(index)))
        print (fmt.format('mu0', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_endmember_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of components", Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_mw(index)))
        print (fmt.format('mu0', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_endmember_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions that access species properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of species", Complex_Solution.cy_Orthopyroxene_Complex_Solution_species_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_species_formula(index)))
        print ("mw: {0:10.2f}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_species_mw(index)))
        print ("Element array:")
        print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_species_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of species", Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_species_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_species_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_species_mw(index)))
        print ("Element array:")
        print (Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_species_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions for molar derivatives
#### First derivative vectors:

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
printLabels(n)
try:
    printResult('dGdn', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass
try:
    printResult('dGdn', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass    

#### The Hessian matrix (molar second derivative matrix) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,nc+1):
    print ("[ ", end=' ')
    for j in range (1,nc+1):
        print ((i,j), end=' ')
    print (']     [', end=' ')
    for j in range (1,nc+1):
        print (model.symmetric_index_from_2d_array(elm=(i,j)), end=' ')
    print (']')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)-1)/2 + len(n))
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d2Gdn2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass
try:
    printResult('d2Gdn2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Complex_Solution.cy_Orthopyroxene_ComplexSolution_calib_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass

#### The 3-D Tensor (molar third derivative tensor) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices:  
If $n_c$ represents the number of components in the solution, and  
if $n_d$ represents the dimensionality of molar derivative (in this case 3), then  
the number of numerically ordered permutations of $n_c$ molar derivatives taken $n_d$ at a time is:

In [ ]:
n_c,n_d = sym.symbols('n_c n_d')
q = sym.factorial(n_c+n_d-1)/sym.factorial(n_d)/sym.factorial(n_c-1)
q

Substituting $n_d$ equal to 3 and simplifying gives:

In [ ]:
q = sym.simplify(q.subs(n_d,3))
q

and, for the number of components in this solution, there will be the following number of unique terms in the third derivative tensor:

In [ ]:
q.subs(n_c,nc)

A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,nc+1):
    for j in range (1,nc+1):
        print ("[", end=' ')
        for k in range (1,nc+1):
            print ("{0:1d}{1:1d}{2:1d}".format(i,j,k), end=' ')
        print ('] ', end=' ')
    print ('  ->  ', end=' ')
    for j in range (1,nc+1):
        print ("[", end=' ')
        for k in range (1,nc+1):
            print (model.symmetric_index_from_3d_array(elm=(i,j,k)), end=' ')
        print ('] ', end=' ')
    print ('')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:10.3e}".format(x), end=' ') for x in result]
    print ("{0:<14.14s}".format(units))
def printLabels(n):
    print ("{0:<15.15s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)+1)*(len(n)+2)/6)
    [print ("[{0:3d}]{1:<5.5s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d3Gdn3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass
try:
    printResult('d3Gdn3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass

## Test and time the generated functions for Feldspar

Time the code

In [ ]:
try:
    %timeit Complex_Solution.cy_Orthopyroxene_Complex_Solution_g(t, p, n)
except AttributeError:
    pass
try:
    %timeit Complex_Solution.cy_Orthopyroxene_Complex_Solution_calib_g(t, p, n) 
except AttributeError:
    pass

Time the Rubicon wrapped Objective-C code

In [ ]:
from thermoengine import model as stdmodel
modelDB = stdmodel.Database()
CpxHC = modelDB.get_phase('Cpx')

In [ ]:
%timeit CpxHC.gibbs_energy(t,p,mol=np.array([1.1, 1.2, 1.3, 0.0, 0.0, 0.0, 0.0])) 

## Methods available only in the "Calib" versions of generated code
### Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
nparam = 0

In [ ]:
try:
    nparam = Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_number()
    names = Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_names()
    units = Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_units()
    values = Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    Complex_Solution.cy_Orthopyroxene_Complex_Solution_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

### Functions that allow modification of a particular parameter value

In [ ]:
try:
    Complex_Solution.cy_Orthopyroxene_Complex_Solution_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Complex_Solution.cy_Orthopyroxene_Complex_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, nparam):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_g(t, p, n, i)))
        print (fmt.format('dGdT', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_dgdt(t, p, n, i)))
        print (fmt.format('dGdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_dgdp(t, p, n, i)))
        print (fmt.format('d2GdT2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d2gdt2(t, p, n, i)))
        print (fmt.format('d2GdTdP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d2gdtdp(t, p, n, i)))
        print (fmt.format('d2GdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d2gdp2(t, p, n, i)))
        print (fmt.format('d3GdT3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d3gdt3(t, p, n, i)))
        print (fmt.format('d3GdT2dP', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d3gdt2dp(t, p, n, i)))
        print (fmt.format('d3GdTdP2', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d3gdtdp2(t, p, n, i)))
        print (fmt.format('d3GdP3', Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_d3gdp3(t, p, n, i)))
except (AttributeError, TypeError):
    pass

### Parameter derivatives of the chemical potential

In [ ]:
def printResult(name, result, units):
    print ("dmu[*]/d {0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<12.12s}".format(units))
def printLabels(n):
    print ("         {0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
try:
    printLabels(n)
    for i in range(0, nparam):
        result = Complex_Solution.cy_Orthopyroxene_Complex_Solution_dparam_dgdn(t,p,n, i)
        printResult(names[i], result, 'J/m^2/p-unit')
except AttributeError:
    pass    